In [8]:
import pandas as pd

df = pd.read_csv('cv-valid-dev-saved.csv')
df['generated_text'] = df['generated_text'].apply(lambda x: str(x).lower())

# Define hot words
hotwords = ["be careful", "destroy", "stranger"]

# Detect hotwords
detected_files = df[df['generated_text'].str.contains('|'.join(hotwords), case=False, na=False)]['filename']

# Save results
with open('detected.txt', 'w') as f:
    for filename in detected_files:
        f.write(f"{filename}\n")

df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration,generated_text
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,be careful with your prognostications said the...
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN,then why should they be surprised when they se...
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN,a young arab also loaded down with baggage ent...
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN,i felt that everything i owned would be destroyed
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN,he moved about invisible but every one could h...


In [10]:
!pip install InstructorEmbedding


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from InstructorEmbedding import INSTRUCTOR
import torch
from tqdm import tqdm
tqdm.pandas()

# Load model
# model = SentenceTransformer('hkunlp/instructor-large')
model = INSTRUCTOR('hkunlp/instructor-large')

# Load dataset
df = pd.read_csv('cv-valid-dev-saved.csv')
df['generated_text'] = df['generated_text'].apply(lambda x: str(x).lower())

# Prepare inputs
instructions = ["Represent the phrase for detecting sensitive commands"]
hotwords = ["be careful", "destroy", "stranger"]
hotword_embeddings = model.encode([[instructions[0], hw] for hw in hotwords], convert_to_tensor=True)

# Function to check similarity
def is_similar(text, threshold=0.6):
    embedding = model.encode([instructions[0], text], convert_to_tensor=True)
    scores = util.cos_sim(embedding, hotword_embeddings)
    return torch.any(scores >= threshold).item()

# Compute similarity
df['similarity'] = df['generated_text'].progress_apply(lambda x: is_similar(x) if pd.notnull(x) else False)

# Save updated file
df.to_csv('cv-valid-dev-with-similarity.csv', index=False)


load INSTRUCTOR_Transformer
max_seq_length  512


100%|██████████| 4076/4076 [03:28<00:00, 19.57it/s]


In [ ]:
df